In [1]:
import sys
import os

SCRIPT_DIR = os.path.dirname(os.path.abspath("..."))
sys.path.append(os.path.dirname(SCRIPT_DIR))
from training.generate import (generate_response, load_model_tokenizer_for_generate, 
                               get_special_token_id, get_special_token_id)
from training.transcript_trainer import PROMPT_FORMAT, create_data_set_from_json_list
from datasets import Dataset, DatasetDict, load_dataset, load_from_disk


/opt/bo_ling/python39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [78]:
file_to_instruction_map = {
    "/opt/home/bo_ling/dataset/michelangelo_so_long.jsonl": "Please answer the following MA helpdesk questions:",
    "/opt/home/bo_ling/dataset/docstrans.jsonl": "Please finish the following doc translation tasks:",
    "/opt/home/bo_ling/dataset/eats_goldenset.jsonl": "Please answer Uber eats products relation questions:"
}
json_file = "/opt/home/bo_ling/dataset/eats_goldenset.jsonl"
dataset_file = json_file.replace(".jsonl", ".hf")
dataset = create_data_set_from_json_list(json_file,
                                         file_to_instruction_map=file_to_instruction_map) 
dataset = dataset.train_test_split(test_size=0.1)
dataset.save_to_disk(dataset_file)

Dataset generator downloaded and prepared to /home/prodadmin/.cache/huggingface/datasets/generator/default-71d75ecb741b2dcb/0.0.0. Subsequent calls will reuse this data.


In [4]:
import json
data = []
max_question_length=2000
max_answer_length=2000
with open("/opt/home/bo_ling/dataset/doc_transcript_pii_data.json") as json_file:
    json_data = json.load(json_file)
for d in json_data:
    try:
        instruction = d['instruction']+":"
        input_text = d["input"][:max_question_length]
        output_text = "\n" + d["output"][:max_answer_length]
        text = PROMPT_FORMAT.format(instruction=instruction,input_text=input_text,output_text=output_text)
        data.append({
                "instruction": instruction,
                "input": input_text, 
                "output": output_text,
                "text": text
            })
    except:
        pass
def gen():
    for d in data:
        yield d
dataset = Dataset.from_generator(gen)
dataset = dataset.train_test_split(test_size=0.1)
dataset.save_to_disk("/opt/home/bo_ling/dataset/doc_transcript_pii_data_simplify.hf")


Generating train split: 0 examples [00:00, ? examples/s]
Generating train split: 21405 examples [00:00, 213216.51 examples/s]
Generating train split: 43000 examples [00:00, 211407.16 examples/s]
Generating train split: 74780 examples [00:00, 210327.20 examples/s]
Generating train split: 96000 examples [00:00, 207296.21 examples/s]
Generating train split: 117000 examples [00:00, 206864.72 examples/s]
Generating train split: 138000 examples [00:00, 205925.12 examples/s]
Generating train split: 159000 examples [00:00, 205540.89 examples/s]
Generating train split: 180000 examples [00:00, 205121.36 examples/s]
Generating train split: 201000 examples [00:00, 205094.32 examples/s]
Generating train split: 222000 examples [00:01, 206168.52 examples/s]
Generating train split: 244000 examples [00:01, 208367.63 examples/s]
Generating train split: 266000 examples [00:01, 209475.40 examples/s]
Generating train split: 288000 examples [00:01, 210317.80 examples/s]
Generating train split: 320000 examp

Dataset generator downloaded and prepared to /home/prodadmin/.cache/huggingface/datasets/generator/default-539c4d3dc8c83bf1/0.0.0. Subsequent calls will reuse this data.



Saving the dataset (0/2 shards):  45%|██████████████████████████████████████████████████████████████████████████▌                                                                                           | 283000/629797 [00:03<00:04, 69591.56 examples/s]


Saving the dataset (1/2 shards):  50%|███████████████████████████████████████████████████████████████████████████████████                                                                                   | 314899/629797 [00:04<00:04, 77324.42 examples/s]
                                                                                                                                                                                                                                                              
Saving the dataset (1/1 shards): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 69978/69978 [00:01<00:00, 64187.55 examples/s]
                                                                                                                                                                                                                                           

In [7]:
new_dataset = load_from_disk("/opt/home/bo_ling/dataset/doc_transcript_pii_data_simplify.hf")
new_dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 629797
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 69978
    })
})

In [8]:
new_dataset['train'][0]

{'instruction': 'Extract issue date from the following input:',
 'input': 'Secretary Hal Taylor Secretary of Law Enforcement DRIVER LICENSE ALABAMA NO.8193988 CLASS D D.O.B. 01-24-1991 EXP 05-13-2024 JUSTIN DEWAYNE JONES 540 FRANCIS PL SW BIRMINGHAM AL 35211-2508 ENDORSEMENTS ISS 05-08-2020 Juste REST SEX M HT 5-05 EYES BRO WT 183 HAIR BLK',
 'output': '\n2020-05-08',
 'text': 'Extract issue date from the following input:\nSecretary Hal Taylor Secretary of Law Enforcement DRIVER LICENSE ALABAMA NO.8193988 CLASS D D.O.B. 01-24-1991 EXP 05-13-2024 JUSTIN DEWAYNE JONES 540 FRANCIS PL SW BIRMINGHAM AL 35211-2508 ENDORSEMENTS ISS 05-08-2020 Juste REST SEX M HT 5-05 EYES BRO WT 183 HAIR BLK\n\n### ### Response:\n:\n2020-05-08\n'}

In [18]:
parquet_file = "/opt/home/bo_ling/dataset/modeling_data_v1.parquet"
import pandas as pd
df = pd.read_parquet(parquet_file, engine='pyarrow')
count = 0

def gen():
    for index, row in df.iterrows():
        yield {"text": row['context_and_msg']}
dataset = Dataset.from_generator(gen)
dataset = dataset.train_test_split(test_size=0.1)
    
dataset.save_to_disk("/opt/home/bo_ling/dataset/modeling_data_v1.hf")



Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 1796 examples [00:00, 17886.03 examples/s]

Generating train split: 4053 examples [00:00, 20631.90 examples/s]

Generating train split: 6371 examples [00:00, 21793.42 examples/s]

Generating train split: 8686 examples [00:00, 22327.86 examples/s]

Generating train split: 11000 examples [00:00, 22468.39 examples/s]

Generating train split: 13360 examples [00:00, 22846.81 examples/s]

Generating train split: 15714 examples [00:00, 23069.83 examples/s]

Generating train split: 19192 examples [00:00, 23077.28 examples/s]

Generating train split: 21533 examples [00:00, 23166.52 examples/s]

Generating train split: 23885 examples [00:01, 23264.08 examples/s]

Generating train split: 27380 examples [00:01, 23230.48 examples/s]

Generating train split: 29736 examples [00:01, 23316.73 examples/s]

Generating train split: 33173 examples [00:01, 23167.25 examples/s]

Generating train split: 35542 examples [00:01, 

Dataset generator downloaded and prepared to /home/prodadmin/.cache/huggingface/datasets/generator/default-7895c5e842d084b9/0.0.0. Subsequent calls will reuse this data.




Saving the dataset (0/1 shards):   0%|                                                                                                                                                                                      | 0/152615 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):  12%|███████████████████▌                                                                                                                                                  | 18000/152615 [00:00<00:00, 173411.10 examples/s]

Saving the dataset (0/1 shards):  26%|███████████████████████████████████████████▌                                                                                                                          | 40000/152615 [00:00<00:00, 195499.43 examples/s]

Saving the dataset (0/1 shards):  41%|███████████████████████████████████████████████████████████████████▍                                                                                                  | 62000/152615 [00:00<00:0

In [29]:
new_dataset = load_from_disk("/opt/home/bo_ling/dataset/modeling_data_v1.hf")
new_dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 152615
    })
    test: Dataset({
        features: ['text'],
        num_rows: 16958
    })
})

In [30]:
new_dataset['train'][0]

{'text': "Specific Information: The restaurant name is Insomnia Cookies (164 Orchard St) and the order value is $19.93. The estimated delivery time is 01:45 and latest arrival time is 02:05. Order status is 6.\n\n###\n\nagent: Hi Charlotte, welcome to the chat support team. I'll be assisting you today. <|endofsentence|>\nagent: May I know your concern please? <|endofsentence|>\nagent: Charlotte, Are we still connected? <|endofsentence|>\nagent: I'm grateful that I was able to help you out today, Charlotte. As I have not heard back from you in a while, I am closing the chat. Please feel free to contact us again for any issues. Thank you for chatting with us today. Stay safe, stay healthy. <|endofsentence|>\nagent: Yes No <|endofsentence|>\nagent: Have we resolved this issue? <|endofsentence|>\n"}

In [31]:
new_dataset['train'][1]

{'text': "Specific Information: The restaurant name is Carlos & Gabbys / Graze Smokehouse / Mexiko (NYC) / Rosa's Coal Fire Chicken  and the order value is $24.07. The estimated delivery time is 19:30 and latest arrival time is 20:00. Order status is 6.\n\n###\n\nAmin: Hello <|endofsentence|>\nAmin: I have a request for my order <|endofsentence|>\nagent: Hi, Amin. Welcome to Chat Support. I'm Archana, and I'm here to help. <|endofsentence|>\nAmin: Can u contact merchant <|endofsentence|>\nagent: I understand your concern for the issue which you have faced. I am sorry for the inconvenience. <|endofsentence|>\nAmin: Can u contact merchant for me with a request <|endofsentence|>\nagent: I can understand your situation, Amin. I know this is inconvenient, but we do not currently provide phone support for these issues right now which is why we are unable to contact the restaurant from our end. <|endofsentence|>\nagent: Have we resolved this issue? <|endofsentence|>\nagent: Yes No <|endofsent

In [9]:
import json 
data = []
max_question_length=2000
max_answer_length=2000
with open("/opt/home/bo_ling/dataset/gds_v4.json") as json_file:
    json_data = json.load(json_file)
for d in json_data:
    try:
        instruction = d['instruction'].strip().replace(":", " input:")
        input_text = d["ocr_text"][:max_question_length]
        output_text = "\n" + d["output"][:max_answer_length]
        text = PROMPT_FORMAT.format(instruction=instruction,input_text=input_text,output_text=output_text)
        data.append({
                "instruction": instruction,
                "input": input_text, 
                "output": output_text,
                "text": text
            })
    except:
        pass
def gen():
    for d in data:
        yield d
dataset = Dataset.from_generator(gen)
dataset.save_to_disk("/opt/home/bo_ling/dataset/gds_v4_simplify.hf")


Generating train split: 0 examples [00:00, ? examples/s]
Generating train split: 21078 examples [00:00, 210032.07 examples/s]
Generating train split: 43000 examples [00:00, 212630.50 examples/s]
Generating train split: 65018 examples [00:00, 214553.18 examples/s]
Generating train split: 87229 examples [00:00, 216758.04 examples/s]
Generating train split: 109207 examples [00:00, 217381.03 examples/s]
Generating train split: 131000 examples [00:00, 216789.70 examples/s]
Generating train split: 153469 examples [00:00, 218790.61 examples/s]
Generating train split: 182662 examples [00:00, 219306.10 examples/s]
                                                                     

Dataset generator downloaded and prepared to /home/prodadmin/.cache/huggingface/datasets/generator/default-0e5ad272234cb383/0.0.0. Subsequent calls will reuse this data.



Saving the dataset (1/1 shards): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 182662/182662 [00:00<00:00, 1219107.15 examples/s]
                                                                                                                                                                                                                                                              

In [10]:
len(dataset)

182662

In [11]:
test_data = load_from_disk("/opt/home/bo_ling/dataset/gds_v4_simplify.hf")
test_data

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 182662
})

In [13]:
test_data[0]

{'instruction': 'Extract dob from the following input:',
 'input': 'California DRIVER LICENSE 081202 LN SOTO MUNGUIA FN ALEC PIERRE 9939 VAN RUITEN ST BELLFLOWER, CA 90706 SEX M AM DONOR USA DLY7679862 EXP 08/12/2025 DOB 08/12/2002 AGE 2 IN 7023 DD 08/04/2021606A3/E5FD/25 CLASS C END NONE RSTR NONE 08122002 HAIR BRN EYES HZL HGT 5\'-11" WGT 205 lb FEDERAL LIMITS APPLY ISS 10/14/2021',
 'output': '\n2002-08-12',
 'text': 'Extract dob from the following input:\nCalifornia DRIVER LICENSE 081202 LN SOTO MUNGUIA FN ALEC PIERRE 9939 VAN RUITEN ST BELLFLOWER, CA 90706 SEX M AM DONOR USA DLY7679862 EXP 08/12/2025 DOB 08/12/2002 AGE 2 IN 7023 DD 08/04/2021606A3/E5FD/25 CLASS C END NONE RSTR NONE 08122002 HAIR BRN EYES HZL HGT 5\'-11" WGT 205 lb FEDERAL LIMITS APPLY ISS 10/14/2021\n\n### ### Response:\n:\n2002-08-12\n'}

In [12]:
count = 0
ins_count = {}
for d in dataset:
    count += 1
    if count >=1000:
        break
    instruction = d['instruction']
    ins_count[instruction] = ins_count.get(instruction, 0) + 1
ins_count   

{'Extract dob from the following input:': 75,
 'Extract issue date from the following input:': 75,
 'Extract expiration date from the following input:': 75,
 'Extract first name from the following input:': 74,
 'Extract middle name from the following input:': 64,
 'Extract last name from the following input:': 74,
 'Extract license class from the following input:': 74,
 'Extract drivers license number from the following input:': 74,
 'Extract zip from the following input:': 74,
 'Extract driving licence issue state from the following input:': 74,
 'Extract address from the following input:': 74,
 'Extract gender from the following input:': 74,
 'Extract city from the following input:': 74,
 'Is the driving license valid for identification?': 44}

In [14]:
dataset[10]

{'instruction': 'Extract address from the following input:',
 'input': 'California DRIVER LICENSE 081202 LN SOTO MUNGUIA FN ALEC PIERRE 9939 VAN RUITEN ST BELLFLOWER, CA 90706 SEX M AM DONOR USA DLY7679862 EXP 08/12/2025 DOB 08/12/2002 AGE 2 IN 7023 DD 08/04/2021606A3/E5FD/25 CLASS C END NONE RSTR NONE 08122002 HAIR BRN EYES HZL HGT 5\'-11" WGT 205 lb FEDERAL LIMITS APPLY ISS 10/14/2021',
 'output': '\n9939 VAN RUITEN ST',
 'text': 'Extract address from the following input:\nCalifornia DRIVER LICENSE 081202 LN SOTO MUNGUIA FN ALEC PIERRE 9939 VAN RUITEN ST BELLFLOWER, CA 90706 SEX M AM DONOR USA DLY7679862 EXP 08/12/2025 DOB 08/12/2002 AGE 2 IN 7023 DD 08/04/2021606A3/E5FD/25 CLASS C END NONE RSTR NONE 08122002 HAIR BRN EYES HZL HGT 5\'-11" WGT 205 lb FEDERAL LIMITS APPLY ISS 10/14/2021\n\n### ### Response:\n:\n9939 VAN RUITEN ST\n'}

In [37]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
)
local_output_dir="/opt/home/doc_transcript_pii_checkpoint-49200"
import torch
tokenizer = AutoTokenizer.from_pretrained(local_output_dir, padding_side="left")
print(len(dataset[10]['text']), len(tokenizer.encode(dataset[10]['text'])))

517 172
